# *QuantNbody* tutorials : visualization

**Chiari Even**

**Laboratoire de Chimie Quantique de Strasbourg, France**

**September 2024**

In this QuantNBody tutorial, we will explore a graphical method to visualize wavefunctions, which can be used to study them or compare them with excited states.

We refer users to the other tutorials to learn how to create many-body Hamiltonians and model systems. Here, we will focus on demonstrating how to use the plotting functions of the QuantNBody package.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse.linalg as la
import quantnbody.fermionic.tools as qnbft
import quantnbody.hybrid_fermionic_bosonic.tools as qnbhbft
import quantnbody.bosonic.tools as qnbbt




## Plot a single wavefunction

To study a wavefunction, we generally focus on the most important coefficients. In the figure below, we plot the ground state of a Fermi-Hubbard Hamiltonian with 3 molecular orbitals containing up to 4 electrons :

In [ ]:

# Parameters for the simulation
nelec_active = 4  # Number of active electrons in the Active-Space  
n_mo = 3  # Number of molecular orbitals  

# Build the many-body basis and operator for the simulation
nbody_basis_ferm =qnbft.build_nbody_basis(n_mo, nelec_active)                      
a_dagger_a =qnbft.build_operator_a_dagger_a(nbody_basis_ferm)

# Hopping terms for the Hamiltonian
h_MO = np.zeros((n_mo, n_mo))
for site in range(n_mo - 1): 
    h_MO[site, site + 1] = h_MO[site + 1, site] = 1

# Interaction terms (U_MO) for the Hamiltonian with U = 0
list_U = [0]  # List of interaction strengths (U)
for U in list_U: 
    U_MO = np.zeros((n_mo, n_mo, n_mo, n_mo))
    for site in range(n_mo):
        U_MO[site, site, site, site] = U
    

# Build the Hamiltonian and solve for eigenvalues and eigenvectors
H = qnbft.build_hamiltonian_fermi_hubbard(h_MO, U_MO, nbody_basis_ferm, a_dagger_a)
eig_en, eig_vec = la.eigsh(H.A)  # Solve for the lowest eigenstates
    
# Plot the wavefunctions with a cutoff for small coefficients
WFT_ferm = eig_vec[:,:]
qnbft.visualize_wft(WFT_ferm[:,0], nbody_basis_ferm)
qnbft.plot_wavefunctions(WFT_ferm, nbody_basis_ferm, list_states=[0], probability=False, cutoff=0.075)





## Plot multiple wavefunctions
    


To compare two or more states, we can plot them simultaneously by specifying the states in the argument list_index=[0,1,2].

In [ ]:

qnbft.visualize_wft(WFT_ferm[:,0], nbody_basis_ferm,cutoff=0.3)


qnbft.plot_wavefunctions(WFT_ferm, nbody_basis_ferm, list_states=[0,1,2], probability=False, cutoff=0.3)



## Bosonic systems


The function works exactly the same for bosonic systems; we just need to give a wavefunction of a bosonic system. Below, we show the wavefunction both in written form and as a plot for the Bose-Hubbard Hamiltonian with 3 modes containing up to 3 bosons.

In [ ]:
n_mode  = 3 # Number of modes
n_boson = 4 # Number of bosons

# Building the one-body tensor in a general extended basis
h_tensor = np.zeros(( n_mode, n_mode )) 
for site in range(n_mode): 
    for site_ in range(n_mode):
        if (site != site_): 
            h_tensor[site,site_] = h_tensor[site_,site] = -1 # <== a lattice fully connected with a same hopping term
            
# Building the two-body tensor in a general extended basis
U_tensor  = np.zeros(( n_mode, n_mode, n_mode, n_mode )) 
for site in range(n_mode):
    U_tensor[ site, site, site, site ]  = - 10.1 # <=== Local on-site attraction of the bosons
    

# Building the many-body basis
nbodybasis_bos = qnbbt.build_nbody_basis( n_mode, n_boson )   

# Building the a†a operators
a_dagger_a = qnbbt.build_operator_a_dagger_a( nbodybasis_bos )  

# Building the matrix representation of the Hamiltonian operators 
Hamiltonian = qnbbt.build_hamiltonian_bose_hubbard( h_tensor,
                                                                U_tensor,
                                                                nbodybasis_bos,
                                                                a_dagger_a ) 
eig_en, eig_vec = la.eigsh( Hamiltonian.A) 

WFT_bos = eig_vec[:,:]

qnbbt.visualize_wft(WFT_bos[:,0],nbodybasis_bos,cutoff=0.3)

qnbbt.plot_wavefunctions(WFT_bos,nbodybasis_bos, list_states=[0,1,3], probability=False, cutoff=0.3)

## Extension to hybrid systems

 

In [ ]:

# ======================================
N_elec = 2  # Number of fermions
N_MO = 2    # Number of molecular orbitals

# ======================================
# Define the bosonic system
# ======================================
N_b_max = 3  # Max bosons in the system
N_mode = 2   # Number of bosonic modes
list_bosons = list(range(N_b_max + 1))  # Possible boson counts per mode

# ==============================================
# Build the hybrid many-body basis and operators
# ==============================================
# Construct the basis and operators
nbody_basis_hyb = qnbhbft.build_nbody_basis(N_mode, list_bosons, N_MO, N_elec)
a_dagger_a = qnbhbft.build_fermion_operator_a_dagger_a(nbody_basis_hyb, N_mode)
b = qnbhbft.build_boson_anihilation_operator_b(nbody_basis_hyb, N_mode)
b_dag = [op.T for op in b]  # Hermitian conjugate of bosonic creation operators

# Define parameters
t_val, U_val, omega_val, lambda_val = 0.5, 1, 0.95, 0.08
Coupling_fermion_boson = [lambda_val] * N_mode

# Fermionic Hamiltonian
t_ = np.zeros((N_MO, N_MO))
U_ = np.zeros((N_MO, N_MO, N_MO, N_MO))
np.fill_diagonal(U_, U_val)
np.fill_diagonal(t_[1:], -t_val)  # Hopping constants

# ====================================
# Bosonic and interaction Hamiltonian
# ====================================
omega_values = [omega_val] * N_mode
h_boson = np.diag(omega_values)

# Build the Holstein Hamiltonian
H_Holstein = qnbhbft.build_hamiltonian_hubbard_holstein(
    t_, U_, a_dagger_a, h_boson, b, Coupling_fermion_boson, nbody_basis_hyb
)

eig_energies, eig_vectors = np.linalg.eigh(H_Holstein.A)

WFT_hyb= eig_vectors[:, :]

# Visualize and plot wavefunctions
qnbhbft.visualize_wft(WFT_hyb[:,0], nbody_basis_hyb, N_mode, cutoff=0.10)
qnbhbft.plot_wavefunctions(WFT_hyb, nbody_basis_hyb, N_mode, list_states=[0, 1, 2], probability=False, cutoff=0.10)


## Optional arguments & Fine-Tunning

Multiple optional arguments are provided in the function to give the user good accessibility to arrange the plot as they want. They will be detailed in this section:

### "List_states"
This argument allows the user to choose the state to plot by specifying the index of the state, e.g., 0 for the ground state, 1 for the first excited state, etc.

### "probability"
Currently, the coefficients of each state are plotted, but the user can choose to plot the probability of the state. The state is given by:

$$ |\Psi_0 \rangle = \sum_i c_i |\phi_i \rangle $$

The probability amplitude of state $i$ is:

$$ |\langle \phi_i |\Psi_0 \rangle|^2 = | c_i |^2 $$



An example is shown below with the previous hybrid system:

In [ ]:
qnbhbft.plot_wavefunctions(WFT_hyb, nbody_basis_hyb,N_mode, list_states=[0], probability=True, cutoff=0.10)



### "cutoff"
To maintain tidiness, the user can choose to plot only the most important states by setting a cutoff value. Only states with values greater than the cutoff will be plotted.

### "label_props"
The size of all labels can be manually adjusted with this parameter. The first element adjusts the size of the bar labels and can be set to "x-small", "small", "medium", "large", "x-large", or "xx-large". The second and third elements adjust the size of the x-axis title and tick labels, respectively.

### "Compact"

The user can get a compact notation for the fermionic state for the numerical print and the graphical plot of the wavefunction. This can be done by setting the argument "compact" to True. 
Instead of writing all the orbital occupancies by "1" if an electron is present and by "0" if it is absent, we only keep the occupied orbitals and specify them by their index. For example,


$$ |01001011\rangle \xrightarrow{} |\overline{1}3 4 \overline{4} \rangle  $$

means the third and fourth spin-alpha orbitals are occupied, while the first and fourth spin-beta orbitals are occupied (indicated by the bar over the index). The second orbital is omitted from the notation since no electron is present in it.


This compact notation is particularly useful for both fermionic and hybrid systems, as it simplifies the representation of fermionic states and allows for more readable outputs in terminal and plotting environments.




In [ ]:
qnbft.visualize_wft(WFT_ferm[:,0], nbody_basis_ferm, cutoff=0.255, compact=False)

qnbft.visualize_wft(WFT_ferm[:,0], nbody_basis_ferm, cutoff=0.255, compact=True)



qnbft.plot_wavefunctions(WFT_ferm, nbody_basis_ferm,
                           probability=False,
                        cutoff=0.10,
                        compact=True
                        )
qnbhbft.plot_wavefunctions(WFT_hyb, nbody_basis_hyb,N_mode,
                           probability=False,
                        cutoff=0.10,
                        compact=True
                        )

